In [1]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("simulated_dataset.csv")  # Load your dataset

In [3]:
df.describe()

,Length_m,Weight_kg_m,Tolerances,Order_Quantity,LME_Price_EUR,Lead_Time_weeks,Quote_Price_SEK
count,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000,950.000000,1000.000000
mean,27.331700,1.329647,0.122532,96386.38800,3.347070,6.996842,3.029540
std,6.703232,0.466752,0.043870,36285.80374,0.448859,3.200861,0.641656
min,15.800000,0.837000,0.050000,32328.00000,2.550000,2.000000,1.890000
25%,21.300000,1.082000,0.085000,65393.50000,2.970000,4.000000,2.480000
50%,27.550000,1.294000,0.120500,96735.00000,3.360000,7.000000,3.020000
75%,33.100000,1.521000,0.161250,127452.00000,3.720000,10.000000,3.580000
max,38.900000,5.166144,0.200000,160864.00000,4.140000,12.000000,4.160000


In [5]:
df = pd.read_csv("quotes_dataset_messy.csv")  # Load your dataset

# Split dataset: rows with and without missing Lead Time
df_lead_train = df[df["Lead Time (weeks)"].notnull()]
df_lead_missing = df[df["Lead Time (weeks)"].isnull()]

# Features to use for prediction (drop leakages and target)
features = ["Alloy", "Finish", "Length", "Weight", "Profile Name", "Tolerances", "GD&T",
            "Order Quantity", "LME price", "Customer Category", "Quote Price (SEK)"]

# Encode categorical variables
encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
X_train = df_lead_train[features].copy()
X_missing = df_lead_missing[features].copy()

X_train[["Alloy", "Finish", "Profile Name", "GD&T", "Customer Category"]] = encoder.fit_transform(
    X_train[["Alloy", "Finish", "Profile Name", "GD&T", "Customer Category"]]
)
X_missing[["Alloy", "Finish", "Profile Name", "GD&T", "Customer Category"]] = encoder.transform(
    X_missing[["Alloy", "Finish", "Profile Name", "GD&T", "Customer Category"]]
)

y_train = df_lead_train["Lead Time (weeks)"]

# Train a random forest regressor
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predict and impute
predicted_lead_time = model.predict(X_missing)
df.loc[df["Lead Time (weeks)"].isnull(), "Lead Time (weeks)"] = predicted_lead_time

# Confirm no missing values remain
missing_after = df["Lead Time (weeks)"].isnull().sum()
missing_after


np.int64(0)

In [6]:
output_file = "quotes_dataset_imputation_done.csv"
df.to_csv(output_file, index=False)